# Lab 6: Training Deep Recurrent Neural Network - Part 2

Nattapat Yuvasuta, 59070501028<br>
Niti Buesamae, 590705010047ID2<br>

**Note: Please name your file**

## Lab Instruction - Language Modelling and Text Classification

In this lab, you will learn to train a deep recurrent neural network using LSTM with the Keras library using the Tensorflow backend. Your task is to implement the natural language modelling and text generation.

Select your favourite book from https://www.gutenberg.org/browse/scores/top and download it as a text file. Then, you will train your language model using RNN-LSTM. 

- Language model (in Thai): http://bit.ly/language_model_1
- Tutorial on how to create a language model (in English): https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

To evaluate the model, the perplexity measurement is used: https://stats.stackexchange.com/questions/10302/what-is-perplexity

Last, fine-tune your model. You have to try different hyperparameter or adding more data. Discuss your result.



**The total lab score is 20 which will be evaluated as follows:**</br>
1. Specification (Do as the instruction said. This include the model tuning section where you have to do a proper amount of tuning) - 10 points
2. Design of logic (No weired things in the process) - 5 points
3. Journaling (Communicate your thought process, comment your code, and discuss result & analyse **in every step**) - 5 points



 #### Import require library

In [70]:
# Import require library
import keras
from keras import *
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
import keras.utils as ku 
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [89]:
from keras.models import load_model

#### 1. Load your data 

In [15]:
# Load data
file = open('data.txt', encoding="utf8")

In [16]:
text = file.readlines()
file.close()

In [17]:
type(text)

list

#### 2. Data Preprocessing 

*Note that only story will be used as a dataset, footnote and creddit are not include.*

The symbol '\n' is indicated the end of the line ``<EOS>``, which is for our model to end the sentence here.

To create a corpus for your model. The following code is can be used:</br>
*Note that other techniques can be used*

```python
# cut the text in semi-redundant sequences of maxlen characters.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
```

The code loop through the data from first word to the last word. The maxlen define a next n word for a model to predict.


In [18]:
#text = [t.replace('\n', '<EOS>') for t in text]

In [19]:
'''maxlen = 1
step = 1
sentences = []
next_chars= []'''

'maxlen = 1\nstep = 1\nsentences = []\nnext_chars= []'

In [20]:
'''# Preprocessing 
# Create corpus & Word vectorization
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])'''

'# Preprocessing \n# Create corpus & Word vectorization\nfor i in range(0, len(text) - maxlen, step):\n    sentences.append(text[i: i + maxlen])\n    next_chars.append(text[i + maxlen])'

In [21]:
data = ''.join(text)

In [22]:
def dataset_preparation():
    pass 
def create_model():
    pass
def generate_text():
    pass

In [23]:
tokenizer = Tokenizer()
def dataset_preparation(data):
    corpus = data.lower().split("\n")    
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences,   
                          maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors,label,max_sequence_len,total_words

ref:  https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

#### 3. Language Model

Define RNN model using LSTM and word embedding representation</br>
We will used perplexity as a metrics

```python
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity
```

To used custom metrics function > https://keras.io/metrics/

For a loss function `categorical_crossentropy` is used, any optimzation method can be applied.

In [91]:
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity

In [15]:
# Define your model

In [15]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(LSTM(500))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['mae', 'acc',perplexity])
    model.fit(predictors, label, epochs=50, verbose=1)
    return model

In [16]:
# Training your model

In [27]:
X, Y, max_len, total_words = dataset_preparation(data)

#### 4. Evaluate your model 

In [17]:
# Evaluate your model using perplexity measurment (You can try adding other measurements as well)

In [16]:
model = create_model(X, Y, max_len, total_words)

Epoch 1/50
96360/96360 [==============================] - 139s 1ms/step - loss: 6.3309 - mean_absolute_error: 2.4749e-04 - acc: 0.0648 - perplexity: 717.1919
Epoch 2/50
96360/96360 [==============================] - 136s 1ms/step - loss: 5.5622 - mean_absolute_error: 2.4254e-04 - acc: 0.1150 - perplexity: 396.1305
Epoch 3/50
96360/96360 [==============================] - 138s 1ms/step - loss: 5.1207 - mean_absolute_error: 2.3899e-04 - acc: 0.1410 - perplexity: 232.7817
Epoch 4/50
96360/96360 [==============================] - 137s 1ms/step - loss: 4.6973 - mean_absolute_error: 2.3560e-04 - acc: 0.1581 - perplexity: 136.9707
Epoch 5/50
96360/96360 [==============================] - 143s 1ms/step - loss: 4.1922 - mean_absolute_error: 2.3095e-04 - acc: 0.1906 - perplexity: 91.0966
Epoch 6/50
96360/96360 [==============================] - 145s 2ms/step - loss: 3.6430 - mean_absolute_error: 2.2228e-04 - acc: 0.2469 - perplexity: 68.3713
Epoch 7/50
96360/96360 [==============================

In [17]:
model.save('model.h5')

#### 5. Text generating

In [ ]:
'''def generate_text(seed_text, max_sequence_len, tolenizer):
    # Loop through the next n words
    for _ in range(200):
        # Preprecess your seed_text and predict the output
        # ======
        # Add your code here
        # ======

        output_word = ""
        for word, index in tokenizer.word_index.items():
            # convert word vector representation to a word string
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        if 
    return seed_text'''

In [26]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

 ref: https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

In [ ]:
# generate your sample text

In [18]:
text = generate_text("Anne", 100, max_len, model)
print(text)

Anne was sitting on the yellow chair by the window gazing mournfully out they morning under always then jane had to him up until her face led marilla’s dinner to the end of been led continued to the pond mercy of the hotel and the night at the house she gets good going to be it how got as i like when you are let me oh marilla i’m willing to think i didn’t know they’re myself but those don’t know i’d like have three head ” she said in a here it’s a chair you can come along isn’t it


#### 6. Model Tuning 

Write down why you design this architecture or why you choose this set of parameter</br>
You should have at least 1 different architectures/set of hyperparameters per person in your team</br>
Last, train your best performed model **on 50 epoch** (or you can try 100 epoch but this will take time)</br>
*Note: For the last step, please turn off a verbose during training

In [ ]:
# Try out different hyperparameter & model architecture

## model tuning #1 
try to use bidirectional rnns that we learn from previuos work (midterm paper) use with LSTM. We think will produce a better result

ref: https://keras.io/layers/wrappers/ <br>
     https://keras.io/getting-started/sequential-model-guide/

In [75]:
def create_model1(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1   
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(Bidirectional(LSTM(200,return_sequences=True)))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['mae', 'acc',perplexity])
    model.fit(predictors, label, epochs=50, verbose=1)
    model.save('model1.h5')
    return model

In [76]:
model1 = create_model1(X, Y, max_len, total_words)

Epoch 1/50
96360/96360 [==============================] - 299s 3ms/step - loss: 6.6928 - mean_absolute_error: 2.4945e-04 - acc: 0.0367 - perplexity: 699.9005
Epoch 2/50
96360/96360 [==============================] - 293s 3ms/step - loss: 6.4023 - mean_absolute_error: 2.4894e-04 - acc: 0.0451 - perplexity: 544.1216
Epoch 3/50
96360/96360 [==============================] - 289s 3ms/step - loss: 6.2466 - mean_absolute_error: 2.4846e-04 - acc: 0.0529 - perplexity: 481.9432
Epoch 4/50
96360/96360 [==============================] - 288s 3ms/step - loss: 6.0485 - mean_absolute_error: 2.4764e-04 - acc: 0.0668 - perplexity: 392.6221
Epoch 5/50
96360/96360 [==============================] - 289s 3ms/step - loss: 5.8775 - mean_absolute_error: 2.4660e-04 - acc: 0.0831 - perplexity: 335.3727
Epoch 6/50
96360/96360 [==============================] - 289s 3ms/step - loss: 5.7278 - mean_absolute_error: 2.4545e-04 - acc: 0.0942 - perplexity: 285.3171
Epoch 7/50
96360/96360 [============================

In [77]:
model1.save('model1.h5')

In [78]:
text1 = generate_text("Anne", 100, max_len, model1)
print(text1)

Anne had been a little sigh of delight was a little little was and willowmere and the brook of the slope and and she and to run in and a and who and i have to and have a thing to have to come to never a of is a white little wood and all it was a very sigh she was the face from the fiddlesticks and at the brook of them for to me i never ” said marilla of that she said to do a very sigh her and if she was fond of that she ” said


## Result of model1 // tuning BLSTM
from observation we found that a perplexity could be lower if we use more epoach on training

## model tuning #2  
from previos model we found that BLSTM is not good so we stick to the original model and improved it by increase droupout layer to 0.5 . we think it might clean some non-important word. 

In [79]:
def create_model2(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(LSTM(500))
    model.add(Dropout(0.5))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['mae', 'acc',perplexity])
    model.fit(predictors, label, epochs=50, verbose=1)
    model.save('model2.h5')
    return model

In [80]:
model2 = create_model2(X, Y, max_len, total_words)

Epoch 1/50
96360/96360 [==============================] - 202s 2ms/step - loss: 6.4376 - mean_absolute_error: 2.4853e-04 - acc: 0.0510 - perplexity: 524.6706
Epoch 2/50
96360/96360 [==============================] - 200s 2ms/step - loss: 5.9084 - mean_absolute_error: 2.4500e-04 - acc: 0.0944 - perplexity: 597.8566
Epoch 3/50
96360/96360 [==============================] - 201s 2ms/step - loss: 5.5489 - mean_absolute_error: 2.4149e-04 - acc: 0.1222 - perplexity: 599.5734
Epoch 4/50
96360/96360 [==============================] - 200s 2ms/step - loss: 5.3063 - mean_absolute_error: 2.3907e-04 - acc: 0.1374 - perplexity: 552.4178
Epoch 5/50
96360/96360 [==============================] - 200s 2ms/step - loss: 5.1026 - mean_absolute_error: 2.3682e-04 - acc: 0.1512 - perplexity: 485.7329
Epoch 6/50
96360/96360 [==============================] - 202s 2ms/step - loss: 4.9027 - mean_absolute_error: 2.3451e-04 - acc: 0.1635 - perplexity: 436.2815
Epoch 7/50
96360/96360 [============================

In [81]:
model2.save('model2.h5')

In [82]:
text2 = generate_text("Anne", 100, max_len, model2)
print(text2)

Anne was sitting on the yellow chair by the window gazing mournfully out of the spare room where the air was a great summer at the present when she was sure of the side of the house was you ” said anne with her arms “i see her a child was about that little and anyhow i thought it was too far to be the same friends i can i was the best of an heart with a boy in the day i left it and i was an orphan and and i had to go to school and do it


## Result of model2 // tuning Dropout
from observation we found that it worsen the result.

## model tuning #3
from previos model we found that add more dropout is not good 
so we have our last method which we add additional layer of LSTM it hope that it will capture more meaningful detail

ref: https://machinelearningmastery.com/stacked-long-short-term-memory-networks/

In [83]:
def create_model3(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(LSTM(500,return_sequences=True))
    model.add(LSTM(500))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['mae', 'acc',perplexity])
    model.fit(predictors, label, epochs=50, verbose=1)
    model.save('model3.h5')
    return model 

In [84]:
model3 = create_model3(X, Y, max_len, total_words)

Epoch 1/50
96360/96360 [==============================] - 376s 4ms/step - loss: 7.2489 - mean_absolute_error: 2.4915e-04 - acc: 0.0309 - perplexity: 5780.1697
Epoch 2/50
96360/96360 [==============================] - 375s 4ms/step - loss: 7.1927 - mean_absolute_error: 2.4915e-04 - acc: 0.0318 - perplexity: 5579.3259
Epoch 3/50
96360/96360 [==============================] - 376s 4ms/step - loss: 7.1914 - mean_absolute_error: 2.4915e-04 - acc: 0.0318 - perplexity: 5551.3724
Epoch 4/50
96360/96360 [==============================] - 375s 4ms/step - loss: 7.1920 - mean_absolute_error: 2.4916e-04 - acc: 0.0311 - perplexity: 5549.5990
Epoch 5/50
96360/96360 [==============================] - 374s 4ms/step - loss: 7.1966 - mean_absolute_error: 2.4917e-04 - acc: 0.0306 - perplexity: 5565.8347
Epoch 6/50
96360/96360 [==============================] - 375s 4ms/step - loss: 7.1966 - mean_absolute_error: 2.4915e-04 - acc: 0.0316 - perplexity: 5574.3256
Epoch 7/50
96360/96360 [======================

In [85]:
model3.save('model3.h5')

In [86]:
text3 = generate_text("Anne", 100, max_len, model3)
print(text3)

Anne and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and


## Result of model3 // stacked LSTM
from observation we found that it worst that pick a random word and rearrange them

#### 7. Help your model to generate a short story 

**Example** https://medium.com/deep-writing/harry-potter-written-by-artificial-intelligence-8a9431803da6

Write your result in a `markdown` cell

# Create your short-story from your model (Add your creativity here)
Anne was sitting on the yellow chair by the window gazing mournfully out they morning under always then jane had to him up until her face led marilla’s dinner to the end of been led continued to the pond mercy of the hotel and the night at the house she gets good going to be it how got as i like when you are let me oh marilla i’m willing to think i didn’t know they’re myself but those don’t know i’d like have three head ” she said in a here it’s a chair you can come along isn’t it


### More on Natural language Processing and Language model
1. https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e 
2. https://medium.com/phrasee/neural-text-generation-generating-text-using-conditional-language-models-a37b69c7cd4b
3. http://karpathy.github.io/2015/05/21/rnn-effectiveness/

**Music generates by RNN**
https://soundcloud.com/optometrist-prime/recurrence-music-written-by-a-recurrent-neural-network
